In [59]:
import numpy as np
from datetime import datetime
import pandas as pd

In [15]:
def bond_price(maturity, coupon_rate, face_value, ytm):
    
    coupon = coupon_rate*face_value

    discount_factors = [1/(1+ytm)**i for i in range(1, maturity+1)]
    cashflows = [coupon]*maturity
    cashflows [-1] += face_value

    return np.array(discount_factors)@np.array(cashflows)

In [ ]:
#Parameters:

maturity = 10
coupon_rate = 0.07
face_value = 1000
ytm = 0.05

bond_price(maturity, coupon_rate, face_value, ytm)

In [83]:
def bond_price_settlement(maturity_date, coupon_rate, face_value, ytm, settlement_date):
    coupon = coupon_rate*face_value

    settlement_date = datetime.strptime(settlement_date, "%d/%m/%Y")
    maturity_date = datetime.strptime(maturity_date, "%d/%m/%Y")
    last_coupon_date = maturity_date.replace(year=settlement_date.year-1)
    next_coupon_date = maturity_date.replace(year=settlement_date.year)

    maturity = maturity_date.year-next_coupon_date.year+1

    t1 = (next_coupon_date-settlement_date).days/365

    time = np.array([t1 + i for i in range(maturity)])
    cashflows = np.array([coupon]*maturity)
    cashflows[-1] += face_value
    discount_factors = 1/(1+ytm)**time

    accrued_interest = (settlement_date - last_coupon_date).days/365 * coupon

    dirty_price = discount_factors@cashflows
    clean_price = dirty_price - accrued_interest

    return dirty_price, clean_price


In [86]:
settlement_date = "13/01/2025"
maturity_date = "25/11/2034"
coupon_rate = 0.03
face_value = 100
ytm = 0.0340

dirty_price, clean_price = bond_price_settlement(maturity_date, coupon_rate, face_value, ytm, settlement_date)

print(f"Dirty price = {dirty_price} \nClean price = {clean_price}")

Dirty price = 97.09134551582721 
Clean price = 96.68860578979982
